In [1]:
import os, time, pandas as pd               # importar librerías externas
from Bio import Entrez
from tqdm import tqdm
import anexo_3_udfs as mf                   # importar funciones propias (UDFS)

print(os.getcwd())                          # comprobar WD
mf.viz()                                    # ampliar visualizaciones de pandas

/Users/pedrofortesgonzalez/Desktop/MASTER_UNIR/31_TFM/3_repo/Anexo_3_tiab_screening


# 1.cargar y de-duplicado

In [2]:
# Cargar CSV de-duplicado
path1 = "./Anexo_3_carpeta_1_inputs/tabla_3.0_articulos_pre_revision.csv"
df = pd.read_csv(path1)
print(df.shape)
print(f"\n{df.columns}")

(103, 90)

Index(['Key', 'Item Type', 'Publication Year', 'Author', 'Title',
       'Publication Title', 'ISBN', 'ISSN', 'DOI', 'Url', 'Abstract Note',
       'Date', 'Date Added', 'Date Modified', 'Access Date', 'Pages',
       'Num Pages', 'Issue', 'Volume', 'Number Of Volumes',
       'Journal Abbreviation', 'Short Title', 'Series', 'Series Number',
       'Series Text', 'Series Title', 'Publisher', 'Place', 'Language',
       'Rights', 'Type', 'Archive', 'Archive Location', 'Library Catalog',
       'Call Number', 'Extra', 'Notes', 'File Attachments', 'Link Attachments',
       'Manual Tags', 'Automatic Tags', 'Editor', 'Series Editor',
       'Translator', 'Contributor', 'Attorney Agent', 'Book Author',
       'Cast Member', 'Commenter', 'Composer', 'Cosponsor', 'Counsel',
       'Interviewer', 'Producer', 'Recipient', 'Reviewed Author',
       'Scriptwriter', 'Words By', 'Guest', 'Number', 'Edition',
       'Running Time', 'Scale', 'Medium', 'Artwork Size', 'Filing Date',
       

Añadir numero de duplicados a stats

### config para screening asistido

In [3]:
# comprobar cuántos artículos con abstract tenemos
col = "Abstract Note"
nonan_values = df[col].value_counts(dropna=True)
nan_values = df[col].value_counts(dropna=False).iloc[0]

print(f"\nNo. papers: {len(df)}")
print(f"Papers w/ abstract in Zotero: {len(nonan_values)}")
print(f"Papers w/out abstract in Zotero: {nan_values}")


No. papers: 103
Papers w/ abstract in Zotero: 97
Papers w/out abstract in Zotero: 6


In [4]:
# Preprocesamiento básico
df['combined_text'] = df['Title'].fillna('') + ' ' + df['Abstract Note'].fillna('')
df['combined_text'] = df['combined_text'].str.lower()

# Inicializar columna de decisiones
df['decision'] = None  # Posibles valores: 'include', 'exclude', None (pendiente)

# Modificación del preprocesamiento para manejar abstracts faltantes
df['Title'] = df['Title'].fillna('')
df['Abstract Note'] = df['Abstract Note'].fillna('')
df['has_abstract'] = ~df['Abstract Note'].str.strip().eq('')

# Crear una bandera para identificar registros sin abstract
print(f"Artículos sin abstract: {sum(~df['has_abstract'])} de {len(df)}")

Artículos sin abstract: 6 de 103


# 2.recuperación automática de abstracts faltantes

## 2.1.recuperación automática de abstracts

In [5]:
# Configuración para PubMed
mail = "pedro.fortes.gonzalez@sergas.es"
Entrez.email = mail

# Recuperar abstracts faltantes usando DOI
abstracts_recovered = 0
articles_without_doi = 0

# Crear una copia de los índices de artículos sin abstract
indices_without_abstract = df[~df['has_abstract']].index.tolist()

# Usar tqdm para mostrar progreso
for idx in tqdm(indices_without_abstract, desc="Recuperando abstracts"):
    doi = df.loc[idx, 'DOI']
    if pd.notnull(doi) and doi.strip() != '':
        abstract = mf.fetch_abstract_by_doi(doi, mail)
        if abstract and len(abstract.strip()) > 0:
            #df.at[idx, 'abstract'] = abstract
            df.at[idx, 'Abstract Note'] = abstract  # Actualizar también la columna original
            df.at[idx, 'has_abstract'] = True
            abstracts_recovered += 1
            time.sleep(0.5)  # Respetar límites de API
    else:
        articles_without_doi += 1

print(f"Abstracts recuperados: {abstracts_recovered}")
print(f"Artículos sin DOI: {articles_without_doi}")
print(f"Artículos que siguen sin abstract: {sum(~df['has_abstract'])}")

# exportar resultado
df.to_csv('Anexo_3_carpeta_1_inputs/tabla_3.0_articulos_pre_revision.csv', index=False)

Recuperando abstracts:   0%|                              | 0/6 [00:00<?, ?it/s]

Error en Unpaywall para DOI 10.1681/ASN.2024xrre28mc: HTTPSConnectionPool(host='api.unpaywall.org', port=443): Max retries exceeded with url: /v2/10.1681/ASN.2024xrre28mc?email=pedro.fortes.gonzalez@sergas.es (Caused by SSLError(SSLEOFError(8, '[SSL: UNEXPECTED_EOF_WHILE_READING] EOF occurred in violation of protocol (_ssl.c:1007)')))


Recuperando abstracts:  17%|███▋                  | 1/6 [00:04<00:20,  4.18s/it]

Error en Semantic Scholar para DOI 10.1681/ASN.2024xrre28mc: 


Recuperando abstracts:  33%|███████▎              | 2/6 [00:06<00:11,  2.86s/it]

...

Error en Semantic Scholar para DOI 10.1681/ASN.20240vzj2pha: Internal Server Error


Recuperando abstracts:  67%|██████████████▋       | 4/6 [00:10<00:04,  2.39s/it]

Error en Semantic Scholar para DOI 10.1681/ASN.2024jkdywd8g: Internal Server Error


Recuperando abstracts: 100%|██████████████████████| 6/6 [00:14<00:00,  2.49s/it]

Error en Semantic Scholar para DOI 10.1016/S0016-5085(17)33807-6: Internal Server Error
Abstracts recuperados: 0
Artículos sin DOI: 0
Artículos que siguen sin abstract: 6


#### ¿cuántos abstracts hemos recuperado en automático?

In [6]:
col = "has_abstract"
print(f"Nº artículos CON abstract:\n{df[col].value_counts(dropna=False)}")

Nº artículos CON abstract:
has_abstract
True     97
False     6
Name: count, dtype: int64


#### 1/7
Voy a intentar recuperarlos manualmente. Para ello, primero separo el df en 2 (con y sin abstract) y los exporto para crear un checkpoint

#### Separar df en df_with_abstract y df_without_abstract
De acuerdo a columna "has_abstract"

In [7]:
# Separar artículos con y sin abstract
df_with_abstract = df[df['has_abstract']].copy()
df_without_abstract = df[~df['has_abstract']].copy()

# Exportar lista de artículos CON abstract para revisión manual (si es necesaria) o recarga
df_with_abstract.to_csv('Anexo_3_carpeta_1_inputs/tabla_3.1_articulos_con_abstract.csv', index=False)
print("Generado 'articulos_con_abstract.csv' para revisión manual")
print(f"Shape: {df_with_abstract.shape}")

# ídem pero para artículos SIN abstract
df_without_abstract.to_csv('Anexo_3_carpeta_1_inputs/tabla_3.2.1_articulos_sin_abstract.csv', index=False)
print("Generado 'articulos_sin_abstract.csv' para revisión manual")
print(f"Shape: {df_without_abstract.shape}")

Generado 'articulos_con_abstract.csv' para revisión manual
Shape: (97, 90)
Generado 'articulos_sin_abstract.csv' para revisión manual
Shape: (6, 90)


## 2.2.Recuperación manual de abstracts

Obtendré sus títulos y haré búsqueda avanzada en WOS y PubMed.

In [8]:
# subsetear cols (para facilitar proceso)
cols = ['Author', 'Title', 'Publication Title', 'DOI', 'Abstract Note']
df_with_abstract = df_with_abstract[df_with_abstract.columns.intersection(cols)]
df_without_abstract = df_without_abstract[df_without_abstract.columns.intersection(cols)]

# almaceno títulos de los papers en lista
noabs_titles = df_without_abstract['Title'].value_counts(dropna=False).index.to_list()

# inicio las queries como string vacía
pm_query = ""
wos_query= ""
# defino variable de longitud total que quiero
total = len(noabs_titles)-1

***
#### query de PubMed

In [9]:
# la construyo con un bucle
for i,t in enumerate(noabs_titles):
    if i == 0:                           # primer término
        pm_query = f"'{t}"
    elif i < total:                      # resto de términos (excp. último)
        pm_query += ("'[tiab] OR '" + t)
    elif i == total:                     # último término
        pm_query += ("'[tiab]")

# resultado
print(f"Total de títulos: {i+1}")
print("\nPubMed Title query:\n")
pm_query

Total de títulos: 6

PubMed Title query:



"'Characterization of GANAB Phenotype in Patients with ADPKD Using Advanced Imaging Biomarkers'[tiab] OR 'URINARY PROTEOMIC BIOMARKERS, A POWERFUL TOOL FOR PROGNOSIS OF AUTOSOMAL DOMINANT POLYCYSTIC KIDNEY DISEASE PROGRESSION'[tiab] OR 'Characterization of IFT140 Phenotype in Patients with ADPKD Using Advanced Imaging Biomarkers'[tiab] OR 'Phenotypic Characterization of Patients with ADPKD with PKD1 and PKD2 Pathogenic Variants Using Advanced Imaging Biomarkers'[tiab] OR 'Addressing the gap in kidney care: A call for algorithmic approach with multiple-biomarker assessment'[tiab]"

Search with this query retrieved 0 results

***
#### query de WOS

In [10]:
# la construyo con un bucle
for i,t in enumerate(noabs_titles):
    if i == 0:                           # primer término
        pm_query = f"TS=('{t}"
    elif i < total:                      # resto de términos (excp. último)
        pm_query += ("' OR '" + t)
    elif i == total:                     # último término
        pm_query += ("')")

# resultado
print(f"Total de títulos: {i+1}")
print("\nWOS Title query:\n")
pm_query

Total de títulos: 6

WOS Title query:



"TS=('Characterization of GANAB Phenotype in Patients with ADPKD Using Advanced Imaging Biomarkers' OR 'URINARY PROTEOMIC BIOMARKERS, A POWERFUL TOOL FOR PROGNOSIS OF AUTOSOMAL DOMINANT POLYCYSTIC KIDNEY DISEASE PROGRESSION' OR 'Characterization of IFT140 Phenotype in Patients with ADPKD Using Advanced Imaging Biomarkers' OR 'Phenotypic Characterization of Patients with ADPKD with PKD1 and PKD2 Pathogenic Variants Using Advanced Imaging Biomarkers' OR 'Addressing the gap in kidney care: A call for algorithmic approach with multiple-biomarker assessment')"

#### unir resultados con df

importar recuperados, hacer concat entre ellos (abs_rec) y pd.merge con df (df, abs_rec, left_join)

In [11]:
# importar recuperados
abs_rec_wos = pd.read_csv("Anexo_3_carpeta_1_inputs/tabla_3.2.2_abs_rec_wos.csv", sep=";")
#abs_rec_pm = pd.read_csv("tabla_3.2.3_abs_rec_pm.csv", sep=";")  # no lo ejecutamos ya que no recuperamos ninguno

# seleccionar subset de columnas
pm_cols = ["AUTHOR", "TITLE", "JOURNAL", "DOI", "ABSTRACT"]
wos_cols = ['AU', 'TI', 'SO', "DL", "AB"]
abs_rec_wos = abs_rec_wos[abs_rec_wos.columns.intersection(wos_cols)]
abs_rec_wos = abs_rec_wos[sorted(abs_rec_wos)]

# renombrar cols
abs_rec_wos.columns = ["ABSTRACT", "AUTHOR", "DOI", "JOURNAL", "TITLE"]
# dropear duplicados
abs_rec_wos = abs_rec_wos.drop_duplicates()

# dropear nas en abstract
abs_rec_wos = abs_rec_wos.dropna(subset=["ABSTRACT"])

## merge both into rec_abs
rec_abs = abs_rec_wos.copy()
# drop articles w/out abstract & duplicates
rec_abs = rec_abs.drop_duplicates(subset=pm_cols[:-1])
rec_abs = rec_abs.dropna(subset=["ABSTRACT"])

# capitalizar columnas del df y renombrar "PUBLICATION TITLE" > "JOURNAL" para facilitar pd.merge()
df_with_abstract.columns = [col.upper() for col in list(df_with_abstract.columns)]
df_with_abstract.columns = ['AUTHOR', 'TITLE', 'JOURNAL',  'DOI', 'ABSTRACT']

## merge df & rec_abs
rec_abs = pd.concat([rec_abs, df_with_abstract])
rec_abs = rec_abs.drop_duplicates(subset="DOI")  # drop articles w/out abstract & duplicates
print(rec_abs.shape)                             # check

(97, 5)


In [12]:
# check
len_i = len(df_with_abstract)
print(f"Artículos iniciales c. abstract: {len_i}")
len_f = len(rec_abs)
recuperados = len_i - len_f
print(f"Artículos recuperados: {recuperados}")
print(f"Artículos finales c. abstract: {len_f}")

Artículos iniciales c. abstract: 97
Artículos recuperados: 0
Artículos finales c. abstract: 97


Hemos recuperado 0 artículos buscando por TÍTULO, busquemos por DOI

In [13]:
# almaceno DOIS de los papers en lista
noabs_dois = df_without_abstract['DOI'].value_counts(dropna=False).index.to_list()
# print
for doi in noabs_dois:
    print(doi)

10.1681/ASN.2024xrre28mc
10.1093/ndt/gfv167.3
10.1681/ASN.20240vzj2pha
10.1681/ASN.2024jkdywd8g
10.4103/jfmpc.jfmpc_1120_24
10.1016/S0016-5085(17)33807-6


- En PM no obtengo nada (papers con otro DOI que dan como resultado este, y son Comentarios)
- En WOS sí que aparecen, pero son 5 Meeting Abstracts y 1 Letter --> los añado a excluidos

In [14]:
# importar stats
stats = pd.read_csv('../Anexo_2_busqueda_bbdd/Anexo_2_carpeta_2_resultados/tabla_2.4_search_stats.csv', index_col=0)

# construir no_papers de deduplicaciones totales como dict (para luego concatenar)
temp_df = {"Not papers": [0, 0, len(noabs_dois), len_f]}
# pasar a df (y trasponer)
temp_df = pd.DataFrame(temp_df).T.astype(int)
# renombrar columnas
temp_df.columns = ["PubMed", "WOS", "Excluidos", "Total"]

# concatenar a stats
stats = pd.concat([stats, temp_df], axis=0)
stats

,PubMed,WOS,Excluidos,Total
#1: términos generales,150112,286496,0,436608
#2: patologías,506457,984086,0,1490543
#3: genes,2914,6025,0,8939
#4: ómicas y biología computacional,2769404,11044214,0,13813618
#5: biomarcadores,1300101,1382132,0,2682233
Registros_identificados,211,49,0,260
Exclusion_cancer,172,45,43,217
Exclusion_last_10y,85,39,93,124
Exclusion_ingles,81,39,4,120
Duplicados y retractados,0,0,17,103


## 2.2.concatenar abstracts recuperados a articulos_con_abstract

In [15]:
# como no hemos recuperado ningún abstract, podemos reasignar df y exportar
df = df[df["has_abstract"]==True]
print(f"Shape: {df.shape}")
df.head()

Shape: (97, 90)


,Key,Item Type,Publication Year,Author,Title,Publication Title,ISBN,ISSN,DOI,Url,Abstract Note,Date,Date Added,Date Modified,Access Date,Pages,Num Pages,Issue,Volume,Number Of Volumes,Journal Abbreviation,Short Title,Series,Series Number,Series Text,Series Title,Publisher,Place,Language,Rights,Type,Archive,Archive Location,Library Catalog,Call Number,Extra,Notes,File Attachments,Link Attachments,Manual Tags,Automatic Tags,Editor,Series Editor,Translator,Contributor,Attorney Agent,Book Author,Cast Member,Commenter,Composer,Cosponsor,Counsel,Interviewer,Producer,Recipient,Reviewed Author,Scriptwriter,Words By,Guest,Number,Edition,Running Time,Scale,Medium,Artwork Size,Filing Date,Application Number,Assignee,Issuing Authority,Country,Meeting Name,Conference Name,Court,References,Reporter,Legal Status,Priority Numbers,Programming Language,Version,System,Code,Code Number,Section,Session,Committee,History,Legislative Body,combined_text,decision,has_abstract
0,REREKIS5,journalArticle,2021,"Stanzick, Kira J.; Li, Yong; Schlosser, Pascal; Gorski, Mathias; Wuttke, Matthias; Thomas, Laurent F.; Rasheed, Humaira; Rowan, Bryce X.; Graham, Sarah E.; Vanderweff, Brett R.; Patil, Snehal B.; Robinson-Cohen, Cassiane; Gaziano, John M.; O'Donnell, Christopher J.; Willer, Cristen J.; Hallan, Stein; Åsvold, Bjørn Olav; Gessner, Andre; Hung, Adriana M.; Pattaro, Cristian; Köttgen, Anna; Stark, Klaus J.; Heid, Iris M.; Winkler, Thomas W.",Discovery and prioritization of variants and genes for kidney function in >1.2 million individuals.,Nature communications,NaN,2041-1723,10.1038/s41467-021-24491-0,NaN,"Genes underneath signals from genome-wide association studies (GWAS) for kidney function are promising targets for functional studies, but prioritizing variants and genes is challenging. By GWAS meta-analysis for creatinine-based estimated glomerular filtration rate (eGFR) from the Chronic Kidney Disease Genetics Consortium and UK Biobank (n = 1,201,909), we expand the number of eGFRcrea loci (424 loci, 201 novel; 9.8% eGFRcrea variance explained by 634 independent signal variants). Our increased sample size in fine-mapping (n = 1,004,040, European) more than doubles the number of signals with resolved fine-mapping (99% credible sets down to 1 variant for 44 signals, ≤5 variants for 138 signals). Cystatin-based eGFR and/or blood urea nitrogen association support 348 loci (n = 460,826 and 852,678, respectively). Our customizable tool for Gene PrioritiSation reveals 23 compelling genes including mechanistic insights and enables navigation through genes and variants likely relevant for kidney function in human to help select targets for experimental follow-up.",2021-07-16,2025-07-29 13:45:51,2025-07-29 13:45:51,NaN,4350,NaN,1,12,NaN,Nat Commun,NaN,NaN,NaN,NaN,NaN,NaN,NaN,eng,© 2021. The Author(s).,NaN,NaN,NaN,NaN,NaN,Place: England PMID: 34272381 PMCID: PMC8285412,NaN,NaN,NaN,"*Genetic Predisposition to Disease; Biomarkers; Creatinine/blood; Cystatins/pharmacology; Databases, Genetic; Europe; Gene Expression Regulation/genetics; Genome-Wide Association Study; Glomerular Filtration Rate/*genetics; Humans; Kidney/*metabolism/physiology; Organ Specificity; Quantitative Trait Loci; Renal Insufficiency, Chronic/genetics/*metabolism; Risk Factors; RNA-Seq; Single-Cell Analysis",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"discovery and prioritization of variants and genes for kidney function in >1.2 million individuals. genes underneath signals from genome-wide association studies (gwas) for kidney function are promising targets for functional studies, but prioritizing variants and genes is challenging. by gwas meta-analysis for creatinine-based estimated glomerular filtration rate (egfr) from the chronic kidney disease genetics consortium and uk biobank (n = 1,201,909), we expand the number of egfrcrea loci (424 loci

# 3.revisar manualmente abstracts no recuperados

In [16]:
# añadir columnas checkbox
cols = ["ADPKD?", "OMICS?", "BIOINFO?", "HUMAN DATA?", "BIOMARKER?", "TRASLATIONAL?", "DECISION", "COMMENT"]
for col in cols[:-4]:
    df[col] = False
# últimas 2 cols
for col in cols[-4:]:
    df[col] = None

# check
df.head(2)

,Key,Item Type,Publication Year,Author,Title,Publication Title,ISBN,ISSN,DOI,Url,Abstract Note,Date,Date Added,Date Modified,Access Date,Pages,Num Pages,Issue,Volume,Number Of Volumes,Journal Abbreviation,Short Title,Series,Series Number,Series Text,Series Title,Publisher,Place,Language,Rights,Type,Archive,Archive Location,Library Catalog,Call Number,Extra,Notes,File Attachments,Link Attachments,Manual Tags,Automatic Tags,Editor,Series Editor,Translator,Contributor,Attorney Agent,Book Author,Cast Member,Commenter,Composer,Cosponsor,Counsel,Interviewer,Producer,Recipient,Reviewed Author,Scriptwriter,Words By,Guest,Number,Edition,Running Time,Scale,Medium,Artwork Size,Filing Date,Application Number,Assignee,Issuing Authority,Country,Meeting Name,Conference Name,Court,References,Reporter,Legal Status,Priority Numbers,Programming Language,Version,System,Code,Code Number,Section,Session,Committee,History,Legislative Body,combined_text,decision,has_abstract,ADPKD?,OMICS?,BIOINFO?,HUMAN DATA?,BIOMARKER?,TRASLATIONAL?,DECISION,COMMENT
0,REREKIS5,journalArticle,2021,"Stanzick, Kira J.; Li, Yong; Schlosser, Pascal; Gorski, Mathias; Wuttke, Matthias; Thomas, Laurent F.; Rasheed, Humaira; Rowan, Bryce X.; Graham, Sarah E.; Vanderweff, Brett R.; Patil, Snehal B.; Robinson-Cohen, Cassiane; Gaziano, John M.; O'Donnell, Christopher J.; Willer, Cristen J.; Hallan, Stein; Åsvold, Bjørn Olav; Gessner, Andre; Hung, Adriana M.; Pattaro, Cristian; Köttgen, Anna; Stark, Klaus J.; Heid, Iris M.; Winkler, Thomas W.",Discovery and prioritization of variants and genes for kidney function in >1.2 million individuals.,Nature communications,NaN,2041-1723,10.1038/s41467-021-24491-0,NaN,"Genes underneath signals from genome-wide association studies (GWAS) for kidney function are promising targets for functional studies, but prioritizing variants and genes is challenging. By GWAS meta-analysis for creatinine-based estimated glomerular filtration rate (eGFR) from the Chronic Kidney Disease Genetics Consortium and UK Biobank (n = 1,201,909), we expand the number of eGFRcrea loci (424 loci, 201 novel; 9.8% eGFRcrea variance explained by 634 independent signal variants). Our increased sample size in fine-mapping (n = 1,004,040, European) more than doubles the number of signals with resolved fine-mapping (99% credible sets down to 1 variant for 44 signals, ≤5 variants for 138 signals). Cystatin-based eGFR and/or blood urea nitrogen association support 348 loci (n = 460,826 and 852,678, respectively). Our customizable tool for Gene PrioritiSation reveals 23 compelling genes including mechanistic insights and enables navigation through genes and variants likely relevant for kidney function in human to help select targets for experimental follow-up.",2021-07-16,2025-07-29 13:45:51,2025-07-29 13:45:51,NaN,4350,NaN,1,12,NaN,Nat Commun,NaN,NaN,NaN,NaN,NaN,NaN,NaN,eng,© 2021. The Author(s).,NaN,NaN,NaN,NaN,NaN,Place: England PMID: 34272381 PMCID: PMC8285412,NaN,NaN,NaN,"*Genetic Predisposition to Disease; Biomarkers; Creatinine/blood; Cystatins/pharmacology; Databases, Genetic; Europe; Gene Expression Regulation/genetics; Genome-Wide Association Study; Glomerular Filtration Rate/*genetics; Humans; Kidney/*metabolism/physiology; Organ Specificity; Quantitative Trait Loci; Renal Insufficiency, Chronic/genetics/*metabolism; Risk Factors; RNA-Seq; Single-Cell Analysis",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"discovery and prioritization of variants and genes for kidney function in >1.2 million individuals. genes underneath signals from genome-wide association studies (gwas) for kidney function are promising targets for functional studies, but prioritizing variants and genes is challenging. by gwas meta-analysis for creatinine-based estimated glomerular filtration rate (egfr) from the chronic kidney disease genetics consortium an

# 4.exportar todo
y empezar screening en siguiente script

In [17]:
df.to_csv('Anexo_3_carpeta_1_inputs/tabla_3.3.1_articulos_para_screening.csv', index=False)
print("Generado 'articulos_para_screening.csv' para screening")
print(f"Shape: {df.shape}")

stats.to_csv('Anexo_3_carpeta_1_inputs/tabla_3.3.2.stats.csv', index=True)
print("Generado 'stats.csv' para screening")
print(f"Shape: {stats.shape}")

Generado 'articulos_para_screening.csv' para screening
Shape: (97, 98)
Generado 'stats.csv' para screening
Shape: (11, 4)
